In [1]:
import sys
import gpytorch
import numpy as np
import pandas as pd
import torch
from datetime import datetime
from scipy.stats import qmc
import itertools
from itertools import combinations_with_replacement, combinations, permutations

import bo_methods_lib
from bo_methods_lib.bo_methods_lib.GPBO_Classes_New import * #Fix this later
from bo_methods_lib.bo_methods_lib.GPBO_Class_fxns import * #Fix this later
from bo_methods_lib.bo_methods_lib.analyze_data import * #Fix this later
from bo_methods_lib.bo_methods_lib.GPBO_Classes_plotters import * #Fix this later
import pympler
import pickle
import signac

from pympler import asizeof

from matplotlib import pyplot as plt

import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 200

#Ignore inconcistent version warning
import warnings
# from sklearn.exceptions import InconsistentVersionWarning
# warnings.filterwarnings(action='ignore', category=InconsistentVersionWarning)
warnings.filterwarnings('ignore')

/afs/crc.nd.edu/user/m/mcarlozo/.conda/envs/Toy_Problem_env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-09-17 16:26:05.430176: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-17 16:26:05.430233: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-17 16:26:05.431693: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-17 16:26:05.439678: I tensorflow/core/platform/cpu_feature_guard

In [30]:
#Set Date and Time
dateTimeObj = datetime.now()
timestampStr = dateTimeObj.strftime("%d-%b-%Y (%H:%M:%S)")
# print("Date and Time: ", timestampStr)
# DateTime = dateTimeObj.strftime("%Y/%m/%d/%H-%M-%S%p")
DateTime = dateTimeObj.strftime("%Y/%m/%d/%H-%M")
DateTime = None ##For Testing

cs_name1  = CS_name_enum(1)
cs_name2  = CS_name_enum(2)

# #Define small case study
# num_x_data = 5
# gen_meth_x = Gen_meth_enum(2)
# num_theta_data1 = 20
# num_theta_data2 = 20
# gen_meth_theta = Gen_meth_enum(1)

# ep0 = 1
# sep_fact = 0.8
# normalize = True
# noise_mean = 0
# noise_std = 0.01
# kernel = Kernel_enum(1)
# lenscl = None
# outputscl = None
# retrain_GP = 1
# seed = 1
# method = GPBO_Methods(Method_name_enum(1)) #1A

#Define cs_params, simulator, and exp_data for CS1
# simulator1 = simulator_helper_test_fxns(cs_name1.value, noise_mean, noise_std, seed)
# exp_data1 = simulator1.gen_exp_data(num_x_data, gen_meth_x)
# sim_data1 = simulator1.gen_sim_data(num_theta_data1, num_x_data, gen_meth_theta, gen_meth_x, sep_fact)
# sim_sse_data1 = simulator1.sim_data_to_sse_sim_data(method, sim_data1, exp_data1, sep_fact)
# val_data1 = simulator1.gen_sim_data(num_theta_data1, num_x_data, gen_meth_theta, gen_meth_x, sep_fact, True)
# val_sse_data1 = simulator1.sim_data_to_sse_sim_data(method, val_data1, exp_data1, sep_fact, True)
# gp_emulator1_s = Type_1_GP_Emulator(sim_sse_data1, val_sse_data1, None, None, None, kernel, lenscl, noise_std, outputscl, retrain_GP, seed, normalize, None, None, None, None)

# #Define cs_params, simulator, and exp_data for CS2
# simulator2 = simulator_helper_test_fxns(cs_name2.value, noise_mean, noise_std, seed)
# exp_data2 = simulator2.gen_exp_data(num_x_data, gen_meth_x)
# sim_data2 = simulator2.gen_sim_data(num_theta_data2, num_x_data, gen_meth_theta, gen_meth_x, sep_fact)
# sim_sse_data2 = simulator2.sim_data_to_sse_sim_data(method, sim_data2, exp_data2, sep_fact)
# val_data2 = simulator2.gen_sim_data(num_theta_data2, num_x_data, gen_meth_theta, gen_meth_x, sep_fact, True)
# val_sse_data2 = simulator2.sim_data_to_sse_sim_data(method, val_data2, exp_data2, sep_fact, True)
# gp_emulator2_s = Type_1_GP_Emulator(sim_sse_data2, val_sse_data2, None, None, None, kernel, lenscl, noise_std, outputscl, retrain_GP, seed, normalize, None, None, None, None)

#Define small case study
num_x_data = 5
gen_meth_x = Gen_meth_enum(2)
num_theta_data1 = 10
num_theta_data2 = 5
gen_meth_theta = Gen_meth_enum(1)

ep0 = 1
sep_fact = 0.8
normalize = True
noise_mean = 0
noise_std = 0.01
kernel = Kernel_enum(1)
lenscl = 1
outputscl = 1
retrain_GP = 0
seed = 1
method = GPBO_Methods(Method_name_enum(5)) #2C

#Define cs_params, simulator, and exp_data for CS1
simulator1 = simulator_helper_test_fxns(cs_name1.value, noise_mean, noise_std, seed)
exp_data1 = simulator1.gen_exp_data(num_x_data, gen_meth_x)
sim_data1 = simulator1.gen_sim_data(num_theta_data1, num_x_data, gen_meth_theta, gen_meth_x, sep_fact)
val_data1 = simulator1.gen_sim_data(num_theta_data1, num_x_data, gen_meth_theta, gen_meth_x, sep_fact, True)
gp_emulator1_e = Type_2_GP_Emulator(sim_data1, val_data1, None, None, None, kernel, lenscl, noise_std, outputscl, retrain_GP, seed, normalize, None, None, None, None)

#Define cs_params, simulator, and exp_data for CS2
simulator2 = simulator_helper_test_fxns(cs_name2.value, noise_mean, noise_std, seed)
exp_data2 = simulator2.gen_exp_data(num_x_data, gen_meth_x)
sim_data2 = simulator2.gen_sim_data(num_theta_data2, num_x_data, gen_meth_theta, gen_meth_x, sep_fact)
val_data2 = simulator2.gen_sim_data(num_theta_data2, num_x_data, gen_meth_theta, gen_meth_x, sep_fact, True)
gp_emulator2_e = Type_2_GP_Emulator(sim_data2, val_data2, None, None, None, kernel, lenscl, noise_std, outputscl, retrain_GP, seed, normalize, None, None, None, None)

gp_emulator = gp_emulator1_e
exp_data = exp_data1
simulator = simulator1
covar = False
train_data, test_data = gp_emulator.set_train_test_data(sep_fact, seed)
gp_emulator.train_gp()
misc_data = Data(None, exp_data.x_vals, None, None,None,None,None,None, simulator.bounds_theta_reg, simulator.bounds_x, sep_fact, seed)
theta = gp_emulator.gp_val_data.theta_vals[0].reshape(1,-1) #Set "candidate thetas"
theta_vals = np.repeat(theta.reshape(1,-1), exp_data.get_num_x_vals() , axis =0)
misc_data.theta_vals = theta_vals #Set misc thetas
feature_misc_data = gp_emulator.featurize_data(misc_data) #Set feature vals
misc_data.gp_mean, misc_data.gp_var = gp_emulator.eval_gp_mean_var_misc(misc_data, feature_misc_data) #Calc mean, var of gp 
sse_mean, sse_var = gp_emulator.eval_gp_sse_var_misc(misc_data, method, exp_data, covar) #Calc mean, var of gp sse
print(sse_mean)
print(sse_var)

╒════════════════════════════════════╤═══════════╤══════════════════╤═════════╤═════════════╤═════════╤═════════╤════════════════════════╕
│ name                               │ class     │ transform        │ prior   │ trainable   │ shape   │ dtype   │ value                  │
╞════════════════════════════════════╪═══════════╪══════════════════╪═════════╪═════════════╪═════════╪═════════╪════════════════════════╡
│ GPR.kernel.kernels[0].variance     │ Parameter │ Softplus         │         │ False       │ ()      │ float64 │ 1.0                    │
├────────────────────────────────────┼───────────┼──────────────────┼─────────┼─────────────┼─────────┼─────────┼────────────────────────┤
│ GPR.kernel.kernels[0].lengthscales │ Parameter │ Softplus         │         │ False       │ (3,)    │ float64 │ [1. 1. 1.]             │
├────────────────────────────────────┼───────────┼──────────────────┼─────────┼─────────────┼─────────┼─────────┼────────────────────────┤
│ GPR.kernel.kernels[1].var

In [ ]:
# #Make GP and Simulator class
results = open_file_helper("workspace/e31ffa2021732474e0e522d4a7a03aee/BO_Results.gz")
#95d031378d09ba17c4c6b58f8ac05a60
#4ad8f582072ab96cee11bc3b6aca30f4

In [ ]:
run = 1
biter = 50
gp_object = copy.copy(results[run-1].list_gp_emulator_class[biter-1])
simulator = copy.copy(results[run-1].simulator_class)
exp_data = copy.copy(results[run-1].exp_data_class)
meth_name = Method_name_enum(results[0].configuration["Method Name Enum Value"])
method = GPBO_Methods(meth_name)
# print(gp_object.train_data_init)
# print(gp_object.noise_std)
# print(simulator.noise_std)
# print(float(scipy.special.polygamma(1, (4*simulator.noise_std**2)/2)))
# print(gp_object.scalerY.lambdas_)

In [ ]:
#Scale training data if necessary
feature_train_data_scaled = gp_object.scalerX.transform(gp_object.feature_train_data)
y_train_data_scaled = gp_object.scalerY.transform(gp_object.train_data.y_vals.reshape(-1,1))
y = y_train_data_scaled
# y = gp_object.train_data.y_vals.reshape(-1,1)

In [ ]:
plt.hist(y, bins=int(len(y)/10), density=True, stacked= True, edgecolor='black')  # Adjust the number of bins as needed
xmin, xmax = plt.xlim() 
mu, std = norm.fit(y[~np.isnan(y)])
x = np.linspace(xmin, xmax, 100) 
p = norm.pdf(x, mu, std) 
plt.plot(x, p, 'r', linewidth=2) 


# Add labels and title
plt.xlabel(r'y^{sim}' +' Value')
plt.ylabel('Probability Density')
plt.title('Histogram of Müller ' + r'$y_0$' ' Data')

In [ ]:
#Evaluate gp_mean over x values
#Generate validation data
val_data = simulator.gen_sim_data(15, exp_data.get_num_x_vals(), Gen_meth_enum(1), Gen_meth_enum(2), 1.0, simulator.seed, False)
if method.emulator == False:
    val_data = simulator.sim_data_to_sse_sim_data(method, val_data, exp_data, 1.0, False)
feat_val = gp_object.featurize_data(val_data)

gp_object.retrain_GP = 25

#Get gp mean
#Change hps of fit gp model or retrain GP
# simulator.noise_std = np.sqrt(float(scipy.special.polygamma(1, (4*simulator.noise_std**2)/2)))
# gp_object.noise_std = None
# new_gp_model.kernel.kernels[0].variance.assign(1.19**2)
# new_gp_model.kernel.kernels[0].lengthscales.assign([4.45,4.45,0.417])
# new_gp_model.kernel.kernels[1].variance.assign(0.0013140374228778637)
# gpflow.utilities.set_trainable(new_gp_model.kernel.kernels[0].variance, False)
# gpflow.utilities.set_trainable(new_gp_model.kernel.kernels[0].lengthscales, False)
# gpflow.utilities.set_trainable(new_gp_model.kernel.kernels[1].variance, False)
gp_object.train_gp()
gpflow.utilities.print_summary(gp_object.fit_gp_model)
# gp_object.fit_gp_model.kernel.kernels[0].variance.assign(1.73)
# gp_object.fit_gp_model.kernel.kernels[0].lengthscales.assign([3.45,3.45])
# gp_object.fit_gp_model.kernel_.k1.k2.length_scale = np.array([3.46, 3.34, 0.708, 3.74, 0.444, 3.5])
# gp_object.fit_gp_model.kernel_.k2.noise_level = 0.0577
# gp_object.fit_gp_model.kernel_.k1.k1.constant_value = 4

misc_gp_mean, misc_var_return = gp_object.eval_gp_mean_var_misc(val_data, feat_val, covar = False)

In [ ]:
# print(misc_var_return)

In [ ]:
print(gp_object.trained_hyperparams)
gpflow.utilities.print_summary(gp_object.fit_gp_model)
#0.976**2 * Matern(length_scale=[3.46, 3.34, 0.708, 3.74, 0.444, 0.464], nu=2.5) + WhiteKernel(noise_level=0.0577)
#0.965**2 * Matern(length_scale=[3.31, 2.83, 0.667, 2.2, 0.365, 0.395], nu=2.5) + WhiteKernel(noise_level=0.00259)

In [ ]:
mse= np.sum((val_data.y_vals - misc_gp_mean)**2)/len(val_data.y_vals)
print("MSE:", mse)
plt.figure()
plt.plot(val_data.y_vals, val_data.y_vals, color='red', alpha=0.7)
plt.scatter(val_data.y_vals, misc_gp_mean, color='blue', alpha=0.7)
plt.errorbar(val_data.y_vals, misc_gp_mean, yerr = 1.96*np.sqrt(abs(misc_var_return)), alpha=0.3, fmt = 'o', color = "blue")
plt.xlabel('True Values')
plt.ylabel('Predicted Values')
plt.title('Parity Plot')
plt.grid(True)
plt.show()